In [16]:
import numpy as np
import pandas as pd

#subjectA = pd.

import scipy.io as sio
raw_data = sio.loadmat('/home/larry/Data/BCI_Competition/IV/BCICIV_1calib_1000Hz_mat/BCICIV_calib_ds1a_1000Hz.mat')


In [17]:
subjectA_recordings = np.array(raw_data['cnt'])
subjectA_cues_raw = raw_data['mrk'][0,0]
sample_rate = raw_data['nfo'][0,0][0][0][0]
cue_length_in_secs = 4
cue_steps = cue_length_in_secs * sample_rate

In [18]:
#np.shape(subjectA_cues)
#print(raw_data.keys())
cues = raw_data["mrk"]
subjectA_cue_times = subjectA_cues_raw[0][0]
subjectA_cue_values = subjectA_cues_raw[1][0]
numCues = len(subjectA_cue_times)

recording_len = np.shape(subjectA_recordings)[0]
targets = np.zeros([recording_len, 3])
cue_steps_remaining = 0
cue_index = 0 #{0:0, -1:1, 1:2}
cue_number = 0
for t in range(recording_len):
    if cue_number < numCues:
        if subjectA_cue_times[cue_number] == t:
            #If new queue is reached
            cue_type = subjectA_cue_values[cue_number]
            if cue_type == -1:
                cue_index = 1
            elif cue_type== 1:
                cue_index = 2
            else:
                raise(exception("unrecognized cue type"))
            cue_steps_remaining = cue_steps-1
            targets[t, cue_index] = 1
            cue_number += 1
        elif cue_steps_remaining > 0:
            #If last queue is still active
            cue_steps_remaining -= 1
            targets[t, cue_index] = 1
        else:
            targets[t, 0] = 1 #Append a value representing no active cue
    else:
            targets[t, 0] = 1 #Append a value representing no active cue

In [136]:
np.shape(subjectA_recordings)
np.shape(targets)
#np.reshape(subjectA_recordings, [-1,64,59])
truncation_length = (recording_len-(recording_len%64))
reshaped_inputs = np.reshape(subjectA_recordings[0:truncation_length,:], [-1,64,59])

(1905940, 3)

In [19]:
#Non-stateful model


from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = np.shape(subjectA_recordings)[1]
timesteps = 64
num_classes = 3
num_epochs = 5
num_lstm_layers = 2

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
for i in range(num_lstm_layers-1):
    model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
#x_train = np.random.random((1000, timesteps, data_dim))
#y_train = np.random.random((1000, num_classes))
truncation_length = (recording_len-(recording_len%timesteps))

reshaped_inputs = np.reshape(subjectA_recordings[0:truncation_length,:], [-1,timesteps,data_dim])
reshaped_targets = np.reshape(targets[0:truncation_length,:], [-1,timesteps,num_classes])

x_train = reshaped_inputs
y_train = reshaped_targets

# Generate dummy validation data
#x_val = np.random.random((100, timesteps, data_dim))
#y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=num_epochs)#,
          #validation_data=(x_val, y_val))

TypeError: Received unknown keyword arguments: {'epochs': 5}

In [6]:
#Stateful model


from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = np.shape(subjectA_recordings)[1]
timesteps = 64
num_classes = 3
batch_size = 32
num_epochs = 5
num_lstm_layers = 2


# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
for i in range(num_lstm_layers-1):
    model.add(LSTM(32, return_sequences=True, stateful=True))
#model.add(LSTM(32, stateful=True))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
#x_train = np.random.random((batch_size * 10, timesteps, data_dim))
#y_train = np.random.random((batch_size * 10, num_classes))
truncation_length = (recording_len-(recording_len%(timesteps*batch_size))) #Might need to make it  (recording_len-(recording_len%(timesteps*batch_size)))

reshaped_inputs = np.reshape(subjectA_recordings[0:truncation_length,:], [-1,timesteps,data_dim])
reshaped_targets = np.reshape(targets[0:truncation_length,:], [-1,timesteps,num_classes])

x_train = reshaped_inputs
y_train = reshaped_targets

# Generate dummy validation data
#x_val = np.random.random((batch_size * 3, timesteps, data_dim))
#y_val = np.random.random((batch_size * 3, num_classes))

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, shuffle=False, epochs=num_epochs)
          #validation_data=(x_val, y_val))
#Next  Previous

Epoch 1/10
11776/29760 [==========>...................] - ETA: 121s - loss: 0.9648 - acc: 0.5751 

In [18]:
all_data_vars = np.concatenate((subjectA_recordings, targets), axis=1)

shifted_data_vars = np.concatenate((np.zeros([1,np.shape(all_data_vars)[1]]),
                                    all_data_vars[:(recording_len-1), :]) , axis=0)

print(shifted_data_vars[1,:])
print(all_data_vars[0,:])
print(np.shape(shifted_data_vars))
print(np.shape(all_data_vars))

[ -97. -100. -133. -192. -131. -150. -136.  -35.  -40. -210. -313.  -94.
 -261.  -70. -203.  -84. -184. -107. -114.  -83. -282. -143.   -2. -135.
  818.  -60. -177.  -99. -196. -189.    2.  -92.   88.  -61. -114.  -94.
 -149. -165.  -52. -154.   60. -103.  -76. -101.  -48. -127.  -34.   64.
   47.  -53.  -45.  -57.  -58. -120.   64.   62.  -26.   74.   28.    1.
    0.    0.]
[ -97. -100. -133. -192. -131. -150. -136.  -35.  -40. -210. -313.  -94.
 -261.  -70. -203.  -84. -184. -107. -114.  -83. -282. -143.   -2. -135.
  818.  -60. -177.  -99. -196. -189.    2.  -92.   88.  -61. -114.  -94.
 -149. -165.  -52. -154.   60. -103.  -76. -101.  -48. -127.  -34.   64.
   47.  -53.  -45.  -57.  -58. -120.   64.   62.  -26.   74.   28.    1.
    0.    0.]
(1905940, 62)
(1905940, 62)


In [19]:
#Autoassociative model

#Stateful model


from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = np.shape(subjectA_recordings)[1]
timesteps = 64
num_classes = 3
batch_size = 32
num_epochs = 5
num_lstm_layers = 2

full_data_dim = data_dim + num_classes


# Expected input batch shape: (batch_size, timesteps, data_dim)
# Note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.
model = Sequential()
model.add(LSTM(32, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, full_data_dim)))
for i in range(num_lstm_layers-1):
    model.add(LSTM(32, return_sequences=True, stateful=True))
#model.add(LSTM(32, stateful=True))
model.add(Dense(full_data_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


all_data_vars = np.concatenate((subjectA_recordings, targets), axis=1)

shifted_data_vars = np.concatenate((np.zeros([1,np.shape(all_data_vars)[1]]),
                                    all_data_vars[:(recording_len-1), :]) , axis=0)


truncation_length = (recording_len-(recording_len%(timesteps*batch_size))) #Might need to make it  (recording_len-(recording_len%(timesteps*batch_size)))

reshaped_inputs = np.reshape(all_data_vars[0:truncation_length,:], [-1,timesteps,full_data_dim])
reshaped_targets = np.reshape(shifted_data_vars[0:truncation_length,:], [-1,timesteps,full_data_dim])

x_train = reshaped_inputs
y_train = reshaped_targets

# Generate dummy validation data
#x_val = np.random.random((batch_size * 3, timesteps, data_dim))
#y_val = np.random.random((batch_size * 3, num_classes))

Using TensorFlow backend.


In [21]:
model.fit(x_train, y_train, batch_size=batch_size, shuffle=False, nb_epoch = num_epochs)


ValueError: Error when checking model input: expected lstm_input_1 to have shape (32, 64, 59) but got array with shape (29760, 64, 62)

In [1]:
#Custom metrics

from keras import metrics

def target_acc(y_true, y_pred, targetCols = (59, 61)):
    tars = y_true[targetCols[0]:targetCols[1]]
    preds = y_pred[targetCols[0]:targetCols[1]]

    return metrics.categorical_accuracy(tars, preds)

Using TensorFlow backend.


In [11]:
import tensorflow as tf
metrics.categorical_accuracy(tf.Variable([0,1,0]),np.Variable([0,1,0]))

AttributeError: 'module' object has no attribute 'Variable'

In [5]:
import keras.backend as K

K.argmax([0,1,0], axis=0)

<tf.Tensor 'ArgMax:0' shape=() dtype=int64>

In [13]:
a = [0,1,2,3]
a[1:2]

[1]

In [21]:
model.fit?